In [263]:
#import cell
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.dates as mdates
from sklearn.externals import joblib
from scipy.signal import medfilt, find_peaks, peak_widths
from astropy.constants import e as charge
import pandas as pd
import sys
import glob
from scipy.ndimage import gaussian_filter1d as gf
sys.path.append('identification/')
sys.path.append('reconnection/')
sys.path.append('pyspace-fork2/analysis/')
import mva
import jets
import event as evt
import models
import crossings_util as Xing
import wind
import postProcess as pp
import make_data
import spacepy.pycdf as pycdf

## Basic function

In [2]:
#function used to plot the raw data of a given spacecraft
def plot_EDR_interval(event, data):
    fig, axarr = plt.subplots(nrows = 9, ncols=1, figsize=(10,15))
    
    axarr[0].plot(data[event.begin:event.end].Bx)
    axarr[1].plot(data[event.begin:event.end].By)
    axarr[2].plot(data[event.begin:event.end].Bz)
    
    axarr[3].plot(data[event.begin:event.end].Vex)
    axarr[4].plot(data[event.begin:event.end].Vey)
    axarr[5].plot(data[event.begin:event.end].Vez)
    
    axarr[6].plot(data[event.begin:event.end].Jx)
    axarr[7].plot(data[event.begin:event.end].Jy)
    axarr[8].plot(data[event.begin:event.end].Jz)
    
    return fig, axarr

In [3]:
#read  the webster EDR list and turn it into an event list

df = pd.read_csv('EDR_intervals.csv', header=None)

df['hours'] = 0
df['minutes'] = 0
df['seconds'] = 0


for i in range(len(df)):
    time = df[4][i].split(sep=':')
    df.hours[i] = float(time[0].strip()[1:])
    df.minutes[i] = float(time[1])
    df.seconds[i] = float(time[2][:-1])

EDR_intervals = []

for i in range(len(df)):
    date = datetime.datetime(df[0][i], df[1][i], df[2][i],df.hours[i], df.minutes[i], df.seconds[i])
    delta = datetime.timedelta(seconds = 40)
    event = evt.Event(date-delta, date+delta)
    EDR_intervals.append(event)

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if s

In [5]:
len(EDR_intervals)

32

In [6]:
spacecrafts = [1,2,3,4]

In [258]:
def get_cdf_file(path, sc, instrument, method, mode, date):
    '''
    For a given spacecraft, instrument and a path where files are stocked, return the list of cdf associated to a certain date
    '''
    
    directory = path+ '/mms{sc}/{instr}/{mode}/l2/{meth}/{yr}/{month:02d}/{day:02d}/'.format(sc=sc, instr=instrument,
                                                                                          mode=mode, meth=method,
                                                                                          yr = date.year,
                                                                                          month = date.month,
                                                                                          day = date.day)
    
    name_file = 'mms{sc}_{instr}_{mode}_l2_{meth}_{yr}{month:02d}{day:02d}{hour:02d}'.format(sc=sc, instr=instrument,
                                                                                          mode=mode, meth=method,
                                                                                          yr = date.year,
                                                                                          month = date.month,
                                                                                          day = date.day,
                                                                                          hour = date.hour)
    
    
    if method is None:
        directory = path+ '/mms{sc}/{instr}/{mode}/l2/{yr}/{month:02d}/{day:02d}/'.format(sc=sc, instr=instrument,
                                                                                          mode=mode,
                                                                                          yr = date.year,
                                                                                          month = date.month,
                                                                                          day = date.day)
    
        name_file = 'mms{sc}_{instr}_{mode}_l2_{yr}{month:02d}{day:02d}{hour:02d}'.format(sc=sc, instr=instrument,
                                                                                          mode=mode,
                                                                                          yr = date.year,
                                                                                          month = date.month,
                                                                                          day = date.day,
                                                                                          hour = date.hour)
    
        
    potential_files = sorted(glob.glob(directory+name_file+'*.cdf'))[::-1]
    

    for file in potential_files:
        if int(file.split('_')[-2][-4:-2]) <= date.minute:
            return file
    
    return None

In [261]:
#get list of cdf for each instrument and sc
path_of_cdf_des1  = [None for i in range(len(EDR_intervals))]
path_of_cdf_des2 = [None for i in range(len(EDR_intervals))]
path_of_cdf_des3 = [None for i in range(len(EDR_intervals))]
path_of_cdf_des4 = [None for i in range(len(EDR_intervals))]



path_of_cdf_dis1 = [None for i in range(len(EDR_intervals))]
path_of_cdf_dis2 = [None for i in range(len(EDR_intervals))]
path_of_cdf_dis3 = [None for i in range(len(EDR_intervals))]
path_of_cdf_dis4 = [None for i in range(len(EDR_intervals))]


path_of_cdf_fgm1 = [None for i in range(len(EDR_intervals))]
path_of_cdf_fgm2 = [None for i in range(len(EDR_intervals))]
path_of_cdf_fgm3 = [None for i in range(len(EDR_intervals))]
path_of_cdf_fgm4 = [None for i in range(len(EDR_intervals))]



for i, cdflist in enumerate([path_of_cdf_des1, path_of_cdf_des2, path_of_cdf_des3, path_of_cdf_des4]):
    for j, event in enumerate(EDR_intervals):
        cdflist[j] = get_cdf_file('/mammouth', i+1, 'fpi', 'des-moms', 'brst', event.begin)
        
        
for i, cdflist in enumerate([path_of_cdf_dis1, path_of_cdf_dis2, path_of_cdf_dis3, path_of_cdf_dis4]):
    for j, event in enumerate(EDR_intervals):
        cdflist[j] = get_cdf_file('/mammouth', i+1, 'fpi', 'dis-moms', 'brst', event.begin)
        
for i, cdflist in enumerate([path_of_cdf_fgm1, path_of_cdf_fgm2, path_of_cdf_fgm3, path_of_cdf_fgm4]):
    for j, event in enumerate(EDR_intervals):
        cdflist[j] = get_cdf_file('/mammouth', i+1, 'fgm', None, 'brst', event.begin)

In [266]:
def make_raw_dataset(index_to_look):
    cdf_des = pycdf.CDF(path_of_cdf_des1[index_to_look])
    cdf_dis = pycdf.CDF(path_of_cdf_dis1[index_to_look])
    cdf_fgm = pycdf.CDF(path_of_cdf_fgm1[index_to_look])



    fpi_des = pd.DataFrame(index = cdf_des['Epoch'][:],
                           data = {'Vex': cdf_des['mms1_des_bulkv_gse_brst'][:, 0],
                                   'Vey': cdf_des['mms1_des_bulkv_gse_brst'][:, 1],
                                   'Vez': cdf_des['mms1_des_bulkv_gse_brst'][:, 2],
                                   'Ne':  cdf_des['mms1_des_numberdensity_brst'][:],
                                   'Temp':cdf_des['mms1_des_tempperp_brst'][:]})

    fpi_dis = pd.DataFrame(index = cdf_dis['Epoch'][:],
                           data = {'Vix':cdf_dis['mms1_dis_bulkv_gse_brst'][:, 0],
                                   'Viy':cdf_dis['mms1_dis_bulkv_gse_brst'][:, 1],
                                   'Viz':cdf_dis['mms1_dis_bulkv_gse_brst'][:, 2],
                                   'Ni': cdf_dis['mms1_dis_numberdensity_brst'][:]})

    fgm = pd.DataFrame(index = cdf_fgm['Epoch'][:],
                       data = {'Bx':cdf_fgm['mms1_fgm_b_gse_brst_l2'][:, 0],
                               'By':cdf_fgm['mms1_fgm_b_gse_brst_l2'][:, 1],
                               'Bz':cdf_fgm['mms1_fgm_b_gse_brst_l2'][:, 2]})

    pos = pd.DataFrame(index = cdf_fgm['Epoch_state'][:],
                      data = {'X':cdf_fgm['mms1_fgm_r_gse_brst_l2'][:, 0],
                              'Y':cdf_fgm['mms1_fgm_r_gse_brst_l2'][:, 1],
                              'Z':cdf_fgm['mms1_fgm_r_gse_brst_l2'][:, 2]})

    dataset1 = make_data.setSameTimeScale([fpi_des, fpi_dis, fgm], ref=0)


    cdf_des = pycdf.CDF(path_of_cdf_des2[index_to_look])
    cdf_dis = pycdf.CDF(path_of_cdf_dis2[index_to_look])
    cdf_fgm = pycdf.CDF(path_of_cdf_fgm2[index_to_look])


    fpi_des = pd.DataFrame(index = cdf_des['Epoch'][:],
                           data = {'Vex': cdf_des['mms2_des_bulkv_gse_brst'][:, 0],
                                   'Vey': cdf_des['mms2_des_bulkv_gse_brst'][:, 1],
                                   'Vez': cdf_des['mms2_des_bulkv_gse_brst'][:, 2],
                                   'Ne':  cdf_des['mms2_des_numberdensity_brst'][:],
                                   'Temp':cdf_des['mms2_des_tempperp_brst'][:]})

    fpi_dis = pd.DataFrame(index = cdf_dis['Epoch'][:],
                           data = {'Vix':cdf_dis['mms2_dis_bulkv_gse_brst'][:, 0],
                                   'Viy':cdf_dis['mms2_dis_bulkv_gse_brst'][:, 1],
                                   'Viz':cdf_dis['mms2_dis_bulkv_gse_brst'][:, 2],
                                   'Ni': cdf_dis['mms2_dis_numberdensity_brst'][:]})

    fgm = pd.DataFrame(index = cdf_fgm['Epoch'][:],
                       data = {'Bx':cdf_fgm['mms2_fgm_b_gse_brst_l2'][:, 0],
                               'By':cdf_fgm['mms2_fgm_b_gse_brst_l2'][:, 1],
                               'Bz':cdf_fgm['mms2_fgm_b_gse_brst_l2'][:, 2]})

    pos = pd.DataFrame(index = cdf_fgm['Epoch_state'][:],
                      data = {'X':cdf_fgm['mms2_fgm_r_gse_brst_l2'][:, 0],
                              'Y':cdf_fgm['mms2_fgm_r_gse_brst_l2'][:, 1],
                              'Z':cdf_fgm['mms2_fgm_r_gse_brst_l2'][:, 2]})

    dataset2 = make_data.setSameTimeScale([fpi_des, fpi_dis, fgm], ref=0)


    cdf_des = pycdf.CDF(path_of_cdf_des3[index_to_look])
    cdf_dis = pycdf.CDF(path_of_cdf_dis3[index_to_look])
    cdf_fgm = pycdf.CDF(path_of_cdf_fgm3[index_to_look])



    fpi_des = pd.DataFrame(index = cdf_des['Epoch'][:],
                           data = {'Vex': cdf_des['mms3_des_bulkv_gse_brst'][:, 0],
                                   'Vey': cdf_des['mms3_des_bulkv_gse_brst'][:, 1],
                                   'Vez': cdf_des['mms3_des_bulkv_gse_brst'][:, 2],
                                   'Ne':  cdf_des['mms3_des_numberdensity_brst'][:],
                                   'Temp':cdf_des['mms3_des_tempperp_brst'][:]})

    fpi_dis = pd.DataFrame(index = cdf_dis['Epoch'][:],
                           data = {'Vix':cdf_dis['mms3_dis_bulkv_gse_brst'][:, 0],
                                   'Viy':cdf_dis['mms3_dis_bulkv_gse_brst'][:, 1],
                                   'Viz':cdf_dis['mms3_dis_bulkv_gse_brst'][:, 2],
                                   'Ni': cdf_dis['mms3_dis_numberdensity_brst'][:]})

    fgm = pd.DataFrame(index = cdf_fgm['Epoch'][:],
                       data = {'Bx':cdf_fgm['mms3_fgm_b_gse_brst_l2'][:, 0],
                               'By':cdf_fgm['mms3_fgm_b_gse_brst_l2'][:, 1],
                               'Bz':cdf_fgm['mms3_fgm_b_gse_brst_l2'][:, 2]})

    pos = pd.DataFrame(index = cdf_fgm['Epoch_state'][:],
                      data = {'X':cdf_fgm['mms3_fgm_r_gse_brst_l2'][:, 0],
                              'Y':cdf_fgm['mms3_fgm_r_gse_brst_l2'][:, 1],
                              'Z':cdf_fgm['mms3_fgm_r_gse_brst_l2'][:, 2]})

    dataset3 = make_data.setSameTimeScale([fpi_des, fpi_dis, fgm], ref=0)



    cdf_des = pycdf.CDF(path_of_cdf_des4[index_to_look])
    cdf_dis = pycdf.CDF(path_of_cdf_dis4[index_to_look])
    cdf_fgm = pycdf.CDF(path_of_cdf_fgm4[index_to_look])


    fpi_des = pd.DataFrame(index = cdf_des['Epoch'][:],
                           data = {'Vex': cdf_des['mms4_des_bulkv_gse_brst'][:, 0],
                                   'Vey': cdf_des['mms4_des_bulkv_gse_brst'][:, 1],
                                   'Vez': cdf_des['mms4_des_bulkv_gse_brst'][:, 2],
                                   'Ne':  cdf_des['mms4_des_numberdensity_brst'][:],
                                   'Temp':cdf_des['mms4_des_tempperp_brst'][:]})

    fpi_dis = pd.DataFrame(index = cdf_dis['Epoch'][:],
                           data = {'Vix':cdf_dis['mms4_dis_bulkv_gse_brst'][:, 0],
                                   'Viy':cdf_dis['mms4_dis_bulkv_gse_brst'][:, 1],
                                   'Viz':cdf_dis['mms4_dis_bulkv_gse_brst'][:, 2],
                                   'Ni': cdf_dis['mms4_dis_numberdensity_brst'][:]})

    fgm = pd.DataFrame(index = cdf_fgm['Epoch'][:],
                       data = {'Bx':cdf_fgm['mms4_fgm_b_gse_brst_l2'][:, 0],
                               'By':cdf_fgm['mms4_fgm_b_gse_brst_l2'][:, 1],
                               'Bz':cdf_fgm['mms4_fgm_b_gse_brst_l2'][:, 2]})

    pos = pd.DataFrame(index = cdf_fgm['Epoch_state'][:],
                      data = {'X':cdf_fgm['mms4_fgm_r_gse_brst_l2'][:, 0],
                              'Y':cdf_fgm['mms4_fgm_r_gse_brst_l2'][:, 1],
                              'Z':cdf_fgm['mms4_fgm_r_gse_brst_l2'][:, 2]})

    dataset4 = make_data.setSameTimeScale([fpi_des, fpi_dis, fgm], ref=0)
    
    for df in [dataset1, dataset2, dataset3, dataset4]:
        df['Jx'] = charge.value*(df.Ni*df.Vix-df.Ne*df.Vex)
        df['Jy'] = charge.value*(df.Ni*df.Viy-df.Ne*df.Vey)
        df['Jz'] = charge.value*(df.Ni*df.Viz-df.Ne*df.Vez)
        
    datasets_sc = [dataset1, dataset2, dataset3, dataset4]

    for i in range(4):
        for name in datasets_sc[i].columns:
            datasets_sc[i].rename(columns={name: name+str(i+1)}, inplace=True)
    
    return datasets_sc

In [272]:
def make_global_dataset(datasets_list):
    ultimate_dataset = make_data.setSameTimeScale(datasets_list, ref=0)
    
    ultimate_dataset['V1'] = np.sqrt(ultimate_dataset.Vex1**2+ultimate_dataset.Vey1**2+ultimate_dataset.Vez1**2)
    ultimate_dataset['V2'] = np.sqrt(ultimate_dataset.Vex2**2+ultimate_dataset.Vey2**2+ultimate_dataset.Vez2**2)
    ultimate_dataset['V3'] = np.sqrt(ultimate_dataset.Vex3**2+ultimate_dataset.Vey3**2+ultimate_dataset.Vez3**2)
    ultimate_dataset['V4'] = np.sqrt(ultimate_dataset.Vex4**2+ultimate_dataset.Vey4**2+ultimate_dataset.Vez4**2)


    ultimate_dataset['B1'] = np.sqrt(ultimate_dataset.Bx1**2+ultimate_dataset.By1**2+ultimate_dataset.Bz1**2)
    ultimate_dataset['B2'] = np.sqrt(ultimate_dataset.Bx2**2+ultimate_dataset.By2**2+ultimate_dataset.Bz2**2)
    ultimate_dataset['B3'] = np.sqrt(ultimate_dataset.Bx3**2+ultimate_dataset.By3**2+ultimate_dataset.Bz3**2)
    ultimate_dataset['B4'] = np.sqrt(ultimate_dataset.Bx4**2+ultimate_dataset.By4**2+ultimate_dataset.Bz4**2)


    ultimate_dataset['J1'] = np.sqrt(ultimate_dataset.Jx1**2+ultimate_dataset.Jy1**2+ultimate_dataset.Jz1**2)
    ultimate_dataset['J2'] = np.sqrt(ultimate_dataset.Jx2**2+ultimate_dataset.Jy2**2+ultimate_dataset.Jz2**2)
    ultimate_dataset['J3'] = np.sqrt(ultimate_dataset.Jx3**2+ultimate_dataset.Jy3**2+ultimate_dataset.Jz3**2)
    ultimate_dataset['J4'] = np.sqrt(ultimate_dataset.Jx4**2+ultimate_dataset.Jy4**2+ultimate_dataset.Jz4**2)
    
    
    ultimate_dataset['std_bx'] = ultimate_dataset[['Bx1', 'Bx2', 'Bx3', 'Bx4']].std(axis=1)
    ultimate_dataset['std_by'] = ultimate_dataset[['By1', 'By2', 'By3', 'By4']].std(axis=1)
    ultimate_dataset['std_bz'] = ultimate_dataset[['Bz1', 'Bz2', 'Bz3', 'Bz4']].std(axis=1)

    ultimate_dataset['std_vx'] = ultimate_dataset[['Vex1', 'Vex2', 'Vex3', 'Vex4']].std(axis=1)
    ultimate_dataset['std_vy'] = ultimate_dataset[['Vey1', 'Vey2', 'Vey3', 'Vey4']].std(axis=1)
    ultimate_dataset['std_vz'] = ultimate_dataset[['Vez1', 'Vez2', 'Vez3', 'Vez4']].std(axis=1)


    ultimate_dataset['std_jx'] = ultimate_dataset[['Jx1', 'Jx2', 'Jx3', 'Jx4']].std(axis=1)
    ultimate_dataset['std_jy'] = ultimate_dataset[['Jy1', 'Jy2', 'Jy3', 'Jy4']].std(axis=1)
    ultimate_dataset['std_jz'] = ultimate_dataset[['Jz1', 'Jz2', 'Jz3', 'Jz4']].std(axis=1)



    ultimate_dataset['std_V'] = ultimate_dataset[['V1', 'V2', 'V3', 'V4']].std(axis=1)
    ultimate_dataset['std_B'] = ultimate_dataset[['B1', 'B2', 'B3', 'B4']].std(axis=1)
    ultimate_dataset['std_J'] = ultimate_dataset[['J1', 'J2', 'J3', 'J4']].std(axis=1)

    return ultimate_dataset

In [269]:
def gaussianfit(serie):

    spec = {
     'time': np.arange(0, len(serie)),
     'y': medfilt(serie, 151),
     'model': [
        {'type': 'GaussianModel'},
        {'type': 'GaussianModel'},
        {'type': 'GaussianModel'},
        {'type': 'GaussianModel'},
        {'type': 'GaussianModel'},
        {'type': 'GaussianModel'}
     ]
    }
    model, params = pp._generate_model(spec)
    output = model.fit(spec['y'], params, x=spec['time'])
    fitted_integral = output.best_fit
    
    return fitted_integral

## make plot for each event

In [276]:
#plot the main physical params and the EDR feature for a given list of intervals

for i, event in enumerate(EDR_intervals):
    raw_datasets = make_raw_dataset(i)
    global_dataset = make_global_dataset(raw_datasets)
    
    df_std = global_dataset[['std_bx', 'std_by', 'std_bz', 'std_vx', 'std_vy', 'std_vz',
                               'std_jx', 'std_jy', 'std_jz', 'std_V', 'std_B', 'std_J']].copy()
    
    
    for name in df_std.columns:
        df_std[name] = gf(df_std[name], 3)
        
    for name in df_std.columns:
        df_std[name] = df_std[name]/df_std[name].max()
        
    EDR_feature = df_std.std_B*df_std.std_V*df_std.std_J
    
    fitted_EDR_feature = pd.Series(index = EDR_feature.index, data = medfilt(EDR_feature, 151))
    
    
    pos = find_peaks(fitted_EDR_feature)[0]
    width = peak_widths(fitted_EDR_feature, pos)
    ref_index = fitted_EDR_feature.index
    EDR_candidates = [evt.Event(ref_index[int(width[2][x])], ref_index[int(width[3][x])]) for x in np.arange(0, len(width[0]))]
    
    
    EDR_candidates = pp.gatherAsEventEnd(EDR_candidates, 5/3600)

    for sc, df in enumerate(raw_datasets):
        fig, axarr = plt.subplots(nrows=5, figsize=(20,25))
        
        axarr[0].plot(df['Ne'+str(sc+1)], color='k')
        
        
        axarr[1].plot(df['Bx'+str(sc+1)], color='k')
        axarr[1].plot(df['By'+str(sc+1)], color='b')
        axarr[1].plot(df['Bz'+str(sc+1)], color='r')
        
        axarr[2].plot(df['Vex'+str(sc+1)], color='k')
        axarr[2].plot(df['Vey'+str(sc+1)], color='b')
        axarr[2].plot(df['Vez'+str(sc+1)], color='r')   
        
        
        axarr[3].plot(df['Jx'+str(sc+1)], color='k')
        axarr[3].plot(df['Jy'+str(sc+1)], color='b')
        axarr[3].plot(df['Jz'+str(sc+1)], color='r') 
                
        axarr[4].plot(fitted_EDR_feature)
        axarr[4].set_ylim(0, 0.13)
        
        for ax in axarr:
            for elt in EDR_candidates:
                ax.axvspan(elt.begin,elt.end, color='g', alpha=0.2, ymax=0.5)

            ax.axvspan(event.begin,event.end, color='r', alpha=0.2, ymin=0.5)

        plt.savefig('EDR/'+str(event.begin)+' '+str(sc+1)+'.png')
        plt.close()

## Detection on unknown intervals

In [277]:
crossings_MMS = evt.read_csv('1hrLists/mms1Crossings.csv', index_col=0, header=0)

In [278]:
crossings_MMS = [x for x in crossings_MMS if x.begin.year==2015]

In [279]:
crossings_MMS = [x for x in crossings_MMS if x.begin.month==10]

In [280]:
len(crossings_MMS)

39

In [192]:


list_of_cdf = []
for i in range(31):
    path = '/mammouth/mms1/fgm/brst/l2/2015/10/{day:02d}/mms'.format(day=i)
    print(path)
    list_of_cdf += glob.glob(path+'*.cdf')

cdfs_to_keep = []


for seum in list_of_cdf:

    name = seum.split('_')[-2]

    date = datetime.datetime(int(name[:4]), int(name[4:6]),int(name[6:8]), int(name[8:10]), int(name[10:12]))
    
    if max(evt.overlapWithList(evt.Event(date, date+datetime.timedelta(seconds=5)), crossings_MMS))>datetime.timedelta(0):
           cdfs_to_keep.append(seum)


EDR_intervals = []

for seum in cdfs_to_keep:
    name = seum.split('_')[-2]
    date = datetime.datetime(int(name[:4]), int(name[4:6]),int(name[6:8]), int(name[8:10]), int(name[10:12]))    
    
    event = evt.Event(date+datetime.timedelta(seconds=15), date+datetime.timedelta(seconds=55))
    EDR_intervals.append(event)

In [ ]:
#get list of cdf for each instrument and sc
path_of_cdf_des1  = [None for i in range(len(EDR_intervals))]
path_of_cdf_des2 = [None for i in range(len(EDR_intervals))]
path_of_cdf_des3 = [None for i in range(len(EDR_intervals))]
path_of_cdf_des4 = [None for i in range(len(EDR_intervals))]



path_of_cdf_dis1 = [None for i in range(len(EDR_intervals))]
path_of_cdf_dis2 = [None for i in range(len(EDR_intervals))]
path_of_cdf_dis3 = [None for i in range(len(EDR_intervals))]
path_of_cdf_dis4 = [None for i in range(len(EDR_intervals))]


path_of_cdf_fgm1 = [None for i in range(len(EDR_intervals))]
path_of_cdf_fgm2 = [None for i in range(len(EDR_intervals))]
path_of_cdf_fgm3 = [None for i in range(len(EDR_intervals))]
path_of_cdf_fgm4 = [None for i in range(len(EDR_intervals))]



for i, cdflist in enumerate([path_of_cdf_des1, path_of_cdf_des2, path_of_cdf_des3, path_of_cdf_des4]):
    for j, event in enumerate(EDR_intervals):
        cdflist[j] = get_cdf_file('/mammouth', i+1, 'fpi', 'des-moms', 'brst', event.begin)
        
        
for i, cdflist in enumerate([path_of_cdf_dis1, path_of_cdf_dis2, path_of_cdf_dis3, path_of_cdf_dis4]):
    for j, event in enumerate(EDR_intervals):
        cdflist[j] = get_cdf_file('/mammouth', i+1, 'fpi', 'dis-moms', 'brst', event.begin)
        
for i, cdflist in enumerate([path_of_cdf_fgm1, path_of_cdf_fgm2, path_of_cdf_fgm3, path_of_cdf_fgm4]):
    for j, event in enumerate(EDR_intervals):
        cdflist[j] = get_cdf_file('/mammouth', i+1, 'fgm', None, 'brst', event.begin)